In [1]:
def query_generator(county_fips, time):
    query = ("""
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX covid-obs-property: <http://covid.geog.ucsb.edu/lod/observedproperty/>
PREFIX covid-instant: <http://covid.geog.ucsb.edu/lod/instant/>
PREFIX covid-place: <http://covid.geog.ucsb.edu/lod/place/>
PREFIX covid: <http://covid.geog.ucsb.edu/lod/ontology/>
PREFIX covid-method: <http://covid.geog.ucsb.edu/lod/method/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>


select ?modelName (min(abs(?predict_value-?groundtruth_value)) as ?min_gap) where {
     
  ?target a covid:Target ;
            sosa:observedProperty covid-obs-property:cum_death ;
            sosa:phenomenonTime ?time ;
            covid:point ?predict_value ;
            sosa:hasFeatureOfInterest ?place ;
            ^sosa:hasMember/^sosa:hasMember/sosa:madeBySensor ?model ;
     .
   
  ?model rdfs:label ?modelName .
            
  ?ground_truth a covid:GroundTruth ;
              sosa:hasFeatureOfInterest ?place ;
              sosa:observedProperty covid-obs-property:cum_death ;
              sosa:phenomenonTime ?time ;
              covid:point ?groundtruth_value .
  
  ?place covid:placeFIPS '%s' .
    
  ?time time:inXSDDateTime '%s' .
}
group by ?modelName
order by ?min_gap
""")% (county_fips, time)
    return query


### This function organizes the queries results from query_generator(model_url, date_str)

def extract_results(result):
    result_list = []
    
    for item in result['results']['bindings']:
        result_list.append([item['modelName']['value'], item['min_gap']['value']])
    
    return result_list

In [3]:
import pandas as pd
state_fips = pd.read_csv("../Question7-Analysis/fips-codes/state_fips_master.csv")
state_fips['fips'] = state_fips['fips'].apply(lambda x: str(x).zfill(2))
state_fips

,state_name,state_abbr,long_name,fips,sumlev,region,division,state,region_name,division_name
0,Alabama,AL,Alabama AL,01,40,3,6,1,South,East South Central
1,Alaska,AK,Alaska AK,02,40,4,9,2,West,Pacific
2,Arizona,AZ,Arizona AZ,04,40,4,8,4,West,Mountain
3,Arkansas,AR,Arkansas AR,05,40,3,7,5,South,West South Central
4,California,CA,California CA,06,40,4,9,6,West,Pacific
5,Colorado,CO,Colorado CO,08,40,4,8,8,West,Mountain
6,Connecticut,CT,Connecticut CT,09,40,1,1,9,Northeast,New England
7,Delaware,DE,Delaware DE,10,40,3,5,10,South,South Atlantic
8,Florida,FL,Florida FL,12,40,3,5,12,South,South Atlantic
9,Georgia,GA,Georgia GA,13,40,3,5,13,South,South Atlantic


## Map on 2021-01-09

In [64]:
import pandas as pd
from pymantic import sparql
import csv
import plotly.express as px
import csv

fips_state_list = state_fips['fips'].tolist()
result_out = []
for item in fips_state_list:
    print("processing county:%s"%item)    
    query_item = query_generator(item, "2021-01-09")
    server = sparql.SPARQLServer('http://128.111.106.227:7201/repositories/Covid-KG')
    result = server.query(query_item)
    result_list = extract_results(result)
    result_pd = pd.DataFrame(result_list, columns =['Model', 'Error'])  
    OW_index = result_pd.index[result_pd['Model'] == 'Karlen-pypm'].tolist()[0]
    OW_ratio = (OW_index+1)/len(result_pd)  # index start from 0, so +1
    result_out.append([item, OW_index+1, len(result_pd), OW_ratio])

processing county:01
processing county:02
processing county:04
processing county:05
processing county:06
processing county:08
processing county:09
processing county:10
processing county:12
processing county:13
processing county:15
processing county:16
processing county:17
processing county:18
processing county:19
processing county:20
processing county:21
processing county:22
processing county:23
processing county:24
processing county:25
processing county:26
processing county:27
processing county:28
processing county:29
processing county:30
processing county:31
processing county:32
processing county:33
processing county:34
processing county:35
processing county:36
processing county:37
processing county:38
processing county:39
processing county:40
processing county:41
processing county:42
processing county:44
processing county:45
processing county:46
processing county:47
processing county:48
processing county:49
processing county:50
processing county:51
processing county:53
processing co

In [65]:
result_out_pd = pd.DataFrame(result_out, columns =['state_fips', 'rank', 'total', 'ratio']) 

In [66]:
result_out_pd

,state_fips,rank,total,ratio
0,01,32,46,0.695652
1,02,4,44,0.090909
2,04,12,46,0.260870
3,05,31,46,0.673913
4,06,18,47,0.382979
5,08,3,46,0.065217
6,09,13,46,0.282609
7,10,4,48,0.083333
8,12,13,48,0.270833
9,13,21,48,0.437500


In [67]:
def convert2class(value):
    if value <=0.1:
        return "Top 10%"
    elif value>0.1 and value<=0.3:
        return "Top 30%"
    elif value>0.3 and value <=0.5:
        return "Top 50%"
    elif value>0.5 and value <=0.7:
        return "Top 70%"
    #elif value>0.7 and value <=0.9:
    #    return "Top 90%"
    #elif value>0. and value <=0.9:
    #    return "Top 80%"
    else:
        return "Top 90%"

In [68]:
result_out_pd['Class'] = result_out_pd['ratio'].apply(lambda row: convert2class(row))

In [69]:
import pandas as pd
state_fips = pd.read_csv("./fips-codes/state_fips_master.csv")
state_fips['fips'] = state_fips['fips'].apply(lambda x: str(x).zfill(2))


state_fips_merged = state_fips.merge(result_out_pd, left_on='fips', right_on='state_fips')

In [70]:
state_fips_merged_sorted = state_fips_merged.sort_values(by=['Class'])

In [71]:
state_fips_merged_sorted

,state_name,state_abbr,long_name,fips,sumlev,region,division,state,region_name,division_name,state_fips,rank,total,ratio,Class
1,Alaska,AK,Alaska AK,02,40,4,9,2,West,Pacific,02,4,44,0.090909,Top 10%
31,New York,NY,New York NY,36,40,1,2,36,Northeast,Middle Atlantic,36,4,49,0.081633,Top 10%
5,Colorado,CO,Colorado CO,08,40,4,8,8,West,Mountain,08,3,46,0.065217,Top 10%
7,Delaware,DE,Delaware DE,10,40,3,5,10,South,South Atlantic,10,4,48,0.083333,Top 10%
26,Nebraska,NE,Nebraska NE,31,40,2,4,31,Midwest,West North Central,31,4,48,0.083333,Top 10%
21,Michigan,MI,Michigan MI,26,40,2,3,26,Midwest,East North Central,26,2,50,0.040000,Top 10%
20,Massachusetts,MA,Massachusetts MA,25,40,1,1,25,Northeast,New England,25,7,49,0.142857,Top 30%
35,Oklahoma,OK,Oklahoma OK,40,40,3,7,40,South,West South Central,40,12,48,0.250000,Top 30%
33,North Dakota,ND,North Dakota ND,38,40,2,4,38,Midwest,West North Central,38,13,48,0.270833,Top 30%
30,New Mexico,NM,New Mexico NM,35,40,4,8,35,West,Mountain,35,10,48,0.208333,Top 30%


In [72]:
fig = px.choropleth(state_fips_merged_sorted, locations='state_abbr', locationmode="USA-states", color='Class', 
                    scope="usa",
                   #color_discrete_sequence = px.colors.qualitative.Light24
                   color_discrete_sequence = ['green', 'lightgreen', 'yellow', 'gold', '#FD3216','' ],
                   )

#fig.update_layout(
#    title_text = 'Where Does OW-Navigator Model Work the Best in the U.S. on Forecasting Cum Death on Jan 9 2021?'
#)
#fig.show()
fig.write_image("Karlen-pypm_state_01092021.png")

## Map on 2021-01-16

In [73]:
import pandas as pd
from pymantic import sparql
import csv
import plotly.express as px
import csv

fips_state_list = state_fips['fips'].tolist()
result_out_01162021 = []
for item in fips_state_list:
    print("processing county:%s"%item)    
    query_item = query_generator(item, "2021-01-16")
    server = sparql.SPARQLServer('http://128.111.106.227:7201/repositories/Covid-KG')
    result = server.query(query_item)
    result_list = extract_results(result)
    result_pd = pd.DataFrame(result_list, columns =['Model', 'Error'])  
    OW_index = result_pd.index[result_pd['Model'] == 'Karlen-pypm'].tolist()[0]
    OW_ratio = (OW_index+1)/len(result_pd)  # index start from 0, so +1
    result_out_01162021.append([item, OW_index+1, len(result_pd), OW_ratio])

processing county:01
processing county:02
processing county:04
processing county:05
processing county:06
processing county:08
processing county:09
processing county:10
processing county:12
processing county:13
processing county:15
processing county:16
processing county:17
processing county:18
processing county:19
processing county:20
processing county:21
processing county:22
processing county:23
processing county:24
processing county:25
processing county:26
processing county:27
processing county:28
processing county:29
processing county:30
processing county:31
processing county:32
processing county:33
processing county:34
processing county:35
processing county:36
processing county:37
processing county:38
processing county:39
processing county:40
processing county:41
processing county:42
processing county:44
processing county:45
processing county:46
processing county:47
processing county:48
processing county:49
processing county:50
processing county:51
processing county:53
processing co

In [74]:
result_out_01162021_pd = pd.DataFrame(result_out_01162021, columns =['state_fips', 'rank', 'total', 'ratio']) 

In [75]:
result_out_01162021_pd

,state_fips,rank,total,ratio
0,01,4,46,0.086957
1,02,20,44,0.454545
2,04,27,46,0.586957
3,05,38,46,0.826087
4,06,5,47,0.106383
5,08,21,46,0.456522
6,09,4,46,0.086957
7,10,21,48,0.437500
8,12,4,48,0.083333
9,13,17,48,0.354167


In [76]:
result_out_01162021_pd['Class'] = result_out_01162021_pd['ratio'].apply(lambda row: convert2class(row))

In [77]:
state_fips_merged_01162021 = state_fips.merge(result_out_01162021_pd, left_on='fips', right_on='state_fips')
state_fips_merged_sorted_01162021 = state_fips_merged_01162021.sort_values(by=['Class'])

In [78]:
state_fips_merged_sorted_01162021

,state_name,state_abbr,long_name,fips,sumlev,region,division,state,region_name,division_name,state_fips,rank,total,ratio,Class
0,Alabama,AL,Alabama AL,01,40,3,6,1,South,East South Central,01,4,46,0.086957,Top 10%
21,Michigan,MI,Michigan MI,26,40,2,3,26,Midwest,East North Central,26,3,49,0.061224,Top 10%
45,Virginia,VA,Virginia VA,51,40,3,5,51,South,South Atlantic,51,2,49,0.040816,Top 10%
42,Texas,TX,Texas TX,48,40,3,7,48,South,West South Central,48,4,50,0.080000,Top 10%
6,Connecticut,CT,Connecticut CT,09,40,1,1,9,Northeast,New England,09,4,46,0.086957,Top 10%
18,Maine,ME,Maine ME,23,40,1,1,23,Northeast,New England,23,3,48,0.062500,Top 10%
8,Florida,FL,Florida FL,12,40,3,5,12,South,South Atlantic,12,4,48,0.083333,Top 10%
40,South Dakota,SD,South Dakota SD,46,40,2,4,46,Midwest,West North Central,46,2,48,0.041667,Top 10%
39,South Carolina,SC,South Carolina SC,45,40,3,5,45,South,South Atlantic,45,3,49,0.061224,Top 10%
31,New York,NY,New York NY,36,40,1,2,36,Northeast,Middle Atlantic,36,4,49,0.081633,Top 10%


In [79]:
fig = px.choropleth(state_fips_merged_sorted_01162021, locations='state_abbr', locationmode="USA-states", color='Class', 
                    scope="usa",
                   #color_discrete_sequence = px.colors.qualitative.Light24
                   color_discrete_sequence = ['green', 'lightgreen', 'yellow', 'gold', '#FD3216','' ],
                   )

#fig.update_layout(
#    title_text = 'Where Does OW-Navigator Model Work the Best in the U.S. on Forecasting Cum Death on Jan 9 2021?'
#)
#fig.show()
fig.write_image("Karlen-pypm_01162021.png")

## Map on 2021-01-02

In [50]:
import pandas as pd
from pymantic import sparql
import csv
import plotly.express as px
import csv

fips_state_list = state_fips['fips'].tolist()
result_out_01022021 = []
for item in fips_state_list:
    print("processing county:%s"%item)    
    query_item = query_generator(item, "2021-01-02")
    server = sparql.SPARQLServer('http://128.111.106.227:7201/repositories/Covid-KG')
    result = server.query(query_item)
    result_list = extract_results(result)
    result_pd = pd.DataFrame(result_list, columns =['Model', 'Error'])  
    OW_index = result_pd.index[result_pd['Model'] == 'OliverWyman-Navigator'].tolist()[0]
    OW_ratio = (OW_index+1)/len(result_pd)  # index start from 0, so +1
    result_out_01022021.append([item, OW_index+1, len(result_pd), OW_ratio])

processing county:01
processing county:02
processing county:04
processing county:05
processing county:06
processing county:08
processing county:09
processing county:10
processing county:12
processing county:13
processing county:15
processing county:16
processing county:17
processing county:18
processing county:19
processing county:20
processing county:21
processing county:22
processing county:23
processing county:24
processing county:25
processing county:26
processing county:27
processing county:28
processing county:29
processing county:30
processing county:31
processing county:32
processing county:33
processing county:34
processing county:35
processing county:36
processing county:37
processing county:38
processing county:39
processing county:40
processing county:41
processing county:42
processing county:44
processing county:45
processing county:46
processing county:47
processing county:48
processing county:49
processing county:50
processing county:51
processing county:53
processing co

In [52]:
result_out_01022021_pd = pd.DataFrame(result_out_01022021, columns =['state_fips', 'rank', 'total', 'ratio']) 

In [60]:
result_out_01022021_pd['Class'] = result_out_01022021_pd['ratio'].apply(lambda row: convert2class(row))

In [61]:
state_fips_merged_01022021 = state_fips.merge(result_out_01022021_pd, left_on='fips', right_on='state_fips')
state_fips_merged_sorted_01022021 = state_fips_merged_01022021.sort_values(by=['Class'])

In [62]:
state_fips_merged_sorted_01022021

,state_name,state_abbr,long_name,fips,sumlev,region,division,state,region_name,division_name,state_fips,rank,total,ratio,Class
2,Arizona,AZ,Arizona AZ,04,40,4,8,4,West,Mountain,04,3,46,0.065217,Top 10%
28,New Hampshire,NH,New Hampshire NH,33,40,1,1,33,Northeast,New England,33,3,48,0.062500,Top 10%
5,Colorado,CO,Colorado CO,08,40,4,8,8,West,Mountain,08,4,46,0.086957,Top 10%
35,Oklahoma,OK,Oklahoma OK,40,40,3,7,40,South,West South Central,40,1,48,0.020833,Top 10%
48,Wisconsin,WI,Wisconsin WI,55,40,2,3,55,Midwest,East North Central,55,3,50,0.060000,Top 10%
10,Hawaii,HI,Hawaii HI,15,40,4,9,15,West,Pacific,15,3,46,0.065217,Top 10%
44,Vermont,VT,Vermont VT,50,40,1,1,50,Northeast,New England,50,10,48,0.208333,Top 30%
31,New York,NY,New York NY,36,40,1,2,36,Northeast,Middle Atlantic,36,13,49,0.265306,Top 30%
30,New Mexico,NM,New Mexico NM,35,40,4,8,35,West,Mountain,35,9,48,0.187500,Top 30%
42,Texas,TX,Texas TX,48,40,3,7,48,South,West South Central,48,7,50,0.140000,Top 30%


In [63]:
fig = px.choropleth(state_fips_merged_sorted_01022021, locations='state_abbr', locationmode="USA-states", color='Class', 
                    scope="usa",
                   #color_discrete_sequence = px.colors.qualitative.Light24
                   color_discrete_sequence = ['green', 'lightgreen', 'yellow', 'gold', '#FD3216','' ],
                   )

#fig.update_layout(
#    title_text = 'Where Does OW-Navigator Model Work the Best in the U.S. on Forecasting Cum Death on Jan 9 2021?'
#)
#fig.show()
fig.write_image("ow_state_01022021.png")